## Class： Logit_BLP

In [11]:
import numpy as np
import pandas as pd
import pyblp

In [12]:
df = pd.read_csv('/Users/terrylu/Desktop/UF/Courses/2025-2026/IO/IO_Code/data/laptop_data.csv')

print(df.head(5))

   market_ids  firm_ids                 model   prices  screen_size_in  \
0           1         1  MacBook Pro 14" (M4)  1606.39            14.0   
1           1         1  MacBook Air 13" (M3)  1091.23            13.3   
2           1         2          Inspiron 14"   700.78            14.0   
3           1         5          VivoBook 15"   636.10            15.6   
4           1         5      ROG Zephyrus 15"  1849.48            15.6   

   ram_gb  storage_gb  brand_Apple  brand_Dell  brand_HP  brand_Lenovo  \
0      16         512         True       False     False         False   
1       8         256         True       False     False         False   
2       8         256        False        True     False         False   
3       8         256        False       False     False         False   
4      16        1000        False       False     False         False   

   os_macOS    shares  nesting_ids  demand_instruments0  demand_instruments1  \
0      True  0.007528         

In [13]:
# prepping data for pyblp
product_data = df.drop(columns=['nesting_ids']).copy() # 我们要先做“简单 Logit”（不是嵌套 Logit），所以不需要分组/巢（nest）信息，先把 nesting_ids 去掉。
# product characteristics, demand, instruments

#simple (agg market share) logit
logit_formulation = pyblp.Formulation(
    "1+ prices + screen_size_in + ram_gb + storage_gb" # 设置消费者的平均效用 δ_jt = β0 + β_priceprice + β1screen + β2ram + β3storage
)
# pyblp.Formulation(...)：用“公式字符串”定义需求方的效用规格（linear characteristics）。
# 语法类似 R/patsy：1 表示截距项；“+ 变量名”加入相应列。
# 注意：默认其实自带截距，"1 + ..." 与 "1+ ..." 都可以，"0 + ..." 才是去掉截距。


problem_logit = pyblp.Problem( # yblp.Problem(formulation, product_data)：构造一个估计问题对象。
    logit_formulation,
    product_data
)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    K1    MD 
---  ----  ---  ----  ----
200  2196   5    5     7  

Formulations:
     Column Indices:         0     1           2           3         4     
--------------------------  ---  ------  --------------  ------  ----------
X1: Linear Characteristics   1   prices  screen_size_in  ram_gb  storage_gb


T = 200

市场的数量（number of markets）。

也就是你有 200 个不同的 market_id，比如不同地区、季度组合。

N = 2196

产品-市场观测值的总数（total product-market observations）。

每一行数据代表某个产品在某个市场的情况。

平均下来 = 2196 ÷ 200 ≈ 11 个产品/市场。

F = 5

产品特征（features）的数量，包括截距（intercept）。

这里是：1 (常数项) + prices + screen_size_in + ram_gb + storage_gb。

K1 = 5

线性效用中的参数数量（linear characteristics）。

和 F 一致，因为你没有额外的 non-linear terms 或 random coefficients。

MD = 7

工具变量和额外矩条件（moment dimensions）的数量。

pyblp 默认会自动生成一些 instruments，比如价格、特征、以及它们的函数。

In [14]:
#estimate logit

results_logit = problem_logit.solve()

Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 1    +8.013112E+01     0      +1.857417E+09      +7.712179E+08  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +1.189330E+03     0      +2.230494E+09      +1.758367E+09  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
       1             prices       screen_size_in       ram_gb         storage_gb   
-----------

why does ram_gb have a negative coefficient?

sicne this may related to brand. for example, apple macbook has low ram but high price.

In [15]:
# logit with dummies (ASUS is baseline)
logit_formulation_2 = pyblp.Formulation(
    "1+ prices + screen_size_in + ram_gb + storage_gb" \
    "+ brand_HP + brand_Apple + brand_Dell + brand_Lenovo"
)

problem_logit_2 = pyblp.Problem(
    logit_formulation_2,
    product_data
)

# ❌ ASUS/Microsoft ≠ outside option。

# ✅ 它们只是基准品牌，效用差异通过其他 dummy 相对估计。

# 真正的 outside option 是单独的一行记录（通常 market share = 1 − Σ inside shares）。

# differnent of baseline and outside option: baseline is just a reference category for categorical variables, while outside option represents the choice of not selecting any of the available products.
# we care compare the utility of different products relative to the baseline, not relative to the outside option.

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    K1    MD 
---  ----  ---  ----  ----
200  2196   5    9     11 

Formulations:
     Column Indices:         0     1           2           3         4             5                 6                 7                  8         
--------------------------  ---  ------  --------------  ------  ----------  --------------  -----------------  ----------------  ------------------
X1: Linear Characteristics   1   prices  screen_size_in  ram_gb  storage_gb  brand_HP[True]  brand_Apple[True]  brand_Dell[True]  brand_Lenovo[True]


In [16]:
results_logit_2 = problem_logit_2.solve()

Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 1    +7.950784E+01     0      +2.653301E+09      +1.315120E+09  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +1.307044E+03     0      +2.875925E+09      +3.872637E+09  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
       1             prices       screen_size_in       ram_gb         storage_gb     brand_HP[T

3.1 消费者价值/支付意愿（WTP）

在简单 Logit 下，某一特征 x 的边际支付意愿：

WTP_x = −β_x / β_price

因为 dU=0 ⇒ β_x dx + β_price dp = 0 ⇒ dp/dx = −β_x/β_price

代入本次估计（单位按你数据的价格单位，多数情况下是“美元”）：

屏幕每 1 英寸：WTP_screen ≈ −0.06049536 / −0.001994877 ≈ 30.3

RAM 每 1 GB：WTP_ram ≈ −0.01725538 / −0.001994877 ≈ 8.65

存储每 1 GB：WTP_storage ≈ −0.001038012 / −0.001994877 ≈ 0.52

品牌溢价（相对基准品牌的效用提升转成钱）：

Apple: ≈ −0.3119381 / −0.001994877 ≈ 156.4

HP: ≈ 102.3；Dell: ≈ 110.2；Lenovo: ≈ 84.1（同理计算）

解读：这是“让消费者保持同等效用时，愿意多付（或少付）多少钱”；正值表示该特征越多越好。

你之前用 diff = +1.725538E−02 / −1.994877E−03 得到 −8.65，本质上也是这个比值，但更标准的表述是“消费者愿意为每增加 1GB RAM 多付约 $8.65”。

In [17]:
diff = +1.725538E-02 / -1.994877E-03
print(diff)  

-8.649846582019844


this diff = +1.725538E-02 / -1.994877E-03 = -8.649846582019844

means:

for one unit increase in ram_gb, the price will decrease by 8.649846582019844 unit, holding other features constant.

In [18]:
results_logit_2.beta
results_logit_2.delta
results_logit_2.xi
results_logit_2.W
# results_logit_2.......

# 输出（保存在 results_logit 对象里，常用属性/方法示例）：
# results_logit.beta：线性参数估计值（含价格系数：应为负）。
# results_logit.delta：估计出来的均值效用 δ̂。
# results_logit.xi：需求误差 ξ̂（解释为产品-市场层面的未观测质量）。
# results_logit.W：GMM 权重矩阵（若走 GMM）。

array([[ 6.14108461e+00,  1.79277597e-03, -2.25501237e-04,
        -4.92434011e+01, -9.38559234e-01, -6.50440115e-02,
         2.53277006e-04,  8.64426066e-02, -1.10853395e+00,
        -4.70334178e-01, -4.53079698e-01],
       [ 1.79277597e-03,  2.60302414e-03, -2.29170064e-03,
         5.70229280e-01, -3.50338655e-02, -4.12146149e-02,
         1.46535725e-04,  3.08764391e-02, -1.16783258e-01,
        -1.47300373e-02,  6.22732727e-02],
       [-2.25501237e-04, -2.29170064e-03,  2.43414364e-03,
         7.54985668e-01, -4.78707193e-02, -5.63288478e-03,
        -1.94424173e-04,  1.36365892e-02, -1.51554808e-01,
        -5.34776247e-02, -9.81648217e-02],
       [-4.92434011e+01,  5.70229280e-01,  7.54985668e-01,
         1.19930793e+04, -7.03265303e+02, -2.23030481e+02,
         8.77845992e-01, -1.87641860e+02, -1.00335597e+03,
        -5.77848131e+02, -3.04431333e+02],
       [-9.38559234e-01, -3.50338655e-02, -4.78707193e-02,
        -7.03265303e+02,  4.52926389e+01,  1.25579663e+01,
  

In [19]:
#compate elasticities
elasticities_logit = results_logit_2.compute_elasticities()

# 函数怎么用
# 来自 pyblp 的结果对象方法，用估计到的参数与当前数据（价格、份额等）计算需求的价格弹性矩阵。
#返回的是一个二维数组 E，形状为 (N, N)，N 是产品-市场观测数。
# 默认计算的是“点弹性”（百分比变化），定义为 ε_jk = ∂s_j/∂p_k × (p_k / s_j)。

# 输出特点
# 对角线元素是 own-price elasticity，应为负数；
# 非对角线是 cross-price elasticity，多为小正数（替代关系）。


Computing elasticities with respect to prices ...
Finished after 00:00:00.



In [20]:
# 矩阵的维度
print(elasticities_logit.shape)

# 前 5 行、前 5 列
print(elasticities_logit[:10, :10])


(2196, 14)
[[-3.18042631  0.02924826  0.03906406  0.03540878  0.02245479  0.04272027
   0.04294751  0.0314521   0.00792754  0.07517989]
 [ 0.02412344 -2.14762088  0.03906406  0.03540878  0.02245479  0.04272027
   0.04294751  0.0314521   0.00792754  0.07517989]
 [ 0.02412344  0.02924826 -1.35890554  0.03540878  0.02245479  0.04272027
   0.04294751  0.0314521   0.00792754  0.07517989]
 [ 0.02412344  0.02924826  0.03906406 -1.2335322   0.02245479  0.04272027
   0.04294751  0.0314521   0.00792754  0.07517989]
 [ 0.02412344  0.02924826  0.03906406  0.03540878 -3.66702951  0.04272027
   0.04294751  0.0314521   0.00792754  0.07517989]
 [ 0.02412344  0.02924826  0.03906406  0.03540878  0.02245479 -0.94885308
   0.04294751  0.0314521   0.00792754  0.07517989]
 [ 0.02412344  0.02924826  0.03906406  0.03540878  0.02245479  0.04272027
  -1.26812526  0.0314521   0.00792754  0.07517989]
 [ 0.02412344  0.02924826  0.03906406  0.03540878  0.02245479  0.04272027
   0.04294751 -2.55821685  0.00792754  0

怎么读这些数值？
1. Own-price elasticity（对角线）

例如第一行第一列：-3.18042631
→ 这是 产品1 对自己价格的弹性。
→ 负值（正常现象），说明提价会导致需求下降。
→ 大小（约 -3.18）表示价格上升 1%，需求下降约 3.18%。

2. Cross-price elasticity（非对角线）

例如第一行第二列：0.0294826
→ 这是 产品1 的需求 对 产品2 价格 的敏感度。
→ 正值 → 说明产品1 和产品2 是 替代品（对方价格上涨 → 自己销量上升）。
→ 如果出现负的交叉项，可能表示互补关系。

3. 范围解释

一般 own-price elasticity 应该 < -1（需求对价格敏感）。

Cross-price elasticity 通常比较小（0.01 ~ 0.1 级别），但能反映竞争关系。

🔹 你的矩阵具体例子

第 2 行第 2 列：-2.14762088
→ 产品2 的 own-price elasticity，大约 -2.15。

第 2 行第 3 列：0.03906406
→ 产品2 的需求对产品3 价格的敏感度（正，替代品）。

第 6 行第 6 列：-0.94885308
→ 产品6 own-price elasticity，约 -0.95（小于 1，说明需求比较缺乏弹性）。

第 9 行第 9 列：-4.81388858
→ 产品9 own-price elasticity，非常有弹性（价格稍微上涨，销量就大幅下降）。

✅ 总结一句话：

对角线（负数）： 每个产品对自己价格的弹性（own-price elasticity）。

非对角线（多为小正数）： 产品之间的替代效应（cross-price elasticity）。

数值大小 → 竞争强度，符号 → 替代还是互补。

In [21]:
np.diag(elasticities_logit)[:5]

array([-3.18042631, -2.14762088, -1.35890554, -1.2335322 , -3.66702951])

In [22]:
# compute Marginal costs
marginal_costs_logit = results_logit_2.compute_costs()
print(marginal_costs_logit[:5])

# 用已估计好的需求系统（来自 Logit 模型的参数和份额），外加一个供给面的行为假设（静态、差异化产品、Bertrand-Nash 竞争、线性定价），
# 通过一阶条件回推出每个产品在每个市场的“边际成本”（marginal cost）。


Computing marginal costs ...


Finished after 00:00:00.

[[1093.51072591]
 [ 578.35072591]
 [ 180.61448052]
 [ 110.36884162]
 [1323.74884162]]


直觉解读：边际成本应低于对应价格，二者差值是加价（markup = price − mc）

### Cereal Data， 除了价格，其他都是产品固定效应

In [23]:
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)

In [24]:
logit_formulation_cereal = pyblp.Formulation('prices',
                                             absorb = 'C(product_ids)' # absorb product fixed effects of each cereal
# pyblp.Formulation(formula, absorb=...): 用公式字符串定义需求方的线性特征（X1），并用 absorb 吸收高维固定效应。
# absorb='C(product_ids)' 的含义是“吸收产品固定效应”，C(...) 是类似 Patsy 的分类变量语法，
# 表示把 product_ids 当作分类哑变量，但不显式生成列，而是用稀疏算法“吸收”掉。

# 'prices' 表示模型里只放价格一个回归变量。

# 在 IO/Logit 里做什么
# 设定 δ_jt = α · price_jt + γ_j + ξ_jt，其中 γ_j 是“产品固定效应”（不随市场变），捕捉未观测、随时间不变的产品质量/偏好（例如品牌口碑、口味、包装等恒定特征）。
# 这样做的目的是控制“时间不变的未观测质量”与价格的相关性，避免把这种相关性错归为价格效应。
)

logit_formulation_cereal

prices + Absorb[C(product_ids)]

In [25]:
problem = pyblp.Problem(logit_formulation_cereal, product_data)
# pyblp.Problem(formulation, product_data)：把模型规格和数据绑定成一个可估计的问题对象。

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    K1    MD    ED 
---  ----  ---  ----  ----  ----
94   2256   5    1     20    1  

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices


In [26]:
logit_results_cereal = problem.solve()

Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +1.899432E+02     0      +6.927228E+07  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 2    +1.874555E+02     0      +5.682065E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices     
---------------
 -3.004710E+01 
(+1.008589E+00)


这两行是在“用估好的模型，算一遍销量份额”，然后“把所有价格整体涨10%，再算一遍”，用来对比涨价前后份额怎么变。

In [33]:
# counterfatual shares
share1 =logit_results_cereal.compute_shares(prices = product_data['prices'])[0:5]
print(f"\n{share1}")
share2 = logit_results_cereal.compute_shares(prices = product_data['prices']*1.1)[0:5]
print(f"\n{share2}")

Computing shares ...
Finished after 00:00:00.


[[0.01241721]
 [0.00780939]
 [0.01299451]
 [0.00576996]
 [0.01793414]]
Computing shares ...
Finished after 00:00:00.


[[0.01163712]
 [0.00644931]
 [0.01015992]
 [0.00453914]
 [0.01310805]]


## 一个nest

In [ ]:
# Nested Logit
product_data1 = product_data.copy()
product_data1['nesting_ids'] = 1  # all cereals in the same nest, the outside option is nest 0
# 用法：新增一列 nesting_ids，并把所有“内部商品”放进同一个巢（编号 1）。
# 含义：这是“内 vs 外”两层结构的嵌套 Logit：所有麦片在同一巢（1），外部选择（不买）是巢 0。ρ>0 时，巢内产品更相似、替代更强，放松简单 Logit 的 IIA。

groups = product_data1.groupby(['market_ids','nesting_ids'])
# 用法：按“市场×巢”分组，便于生成巢层面的变量。
# 含义：Nested Logit 里常需要巢内统计量（如巢内份额、品类数量）来做解释或作为工具变量。

product_data1['demand_instruments20'] = groups['shares'].transform(np.size) # number of products in each nest within each market
# 含义：对按 ['market_ids','nesting_ids'] 分组后的每个组，计算该组行数（包括 NaN），
# 并把“组大小”广播回每一行。返回的是与原 DataFrame 同长度、索引对齐的一列。

# groups = product_data1.groupby(['market_ids','nesting_ids'])
        # 含义：按两个键把表分组。每个“市场×巢”的组合是一组。
        # 返回：一个“分组对象”，本身不计算，等你后面调用聚合/变换时才算。
# product_data1['demand_instruments20'] = groups['shares'].transform(np.size)
        # 含义：对每组的 shares 列执行 np.size（组内元素个数），并把“该组的结果”广播回组内每一行。 随便挑一列都一样，因为组大小对该组内所有行都一样。
        # 结果：新增一列，每行是“它所属组的行数”。行数不变、索引对齐，能直接赋回原表。
        # 等价写法：groups['shares'].transform('size')、groups['shares'].transform(len)

# 用法：在每个“市场×巢”里计算该巢的产品数量，并把结果回填到每行。
# 含义：把“巢内产品个数”作为一个需求侧工具变量（demand_instruments…），
# 帮助识别价格系数/ρ（直觉：同类产品多、竞争激烈，会影响巢的“吸引力”）；前提是该数量对价格外生。


nl_formaulation = pyblp.Formulation('0 + prices')
problem = pyblp.Problem(nl_formaulation, product_data1)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    K1    MD    H 
---  ----  ---  ----  ----  ---
94   2256   5    1     21    1 

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices


Initializing the problem ... / Initialized the problem after ...

只是状态信息：问题对象创建并完成初始化，用了多少时间。
Dimensions 表

T = 94

数据里 market_ids 的去重个数（多少个市场）。等价于 product_data['market_ids'].nunique()。

N = 2256

行数（产品×市场观测的总行数）。等价于 len(product_data)。

F = 5

内部“特征矩阵”的列数（pyblp 构造特征/工具变量时用到的列数）。注意它不一定等于 K1，尤其当你吸收了固定效应或开了额外特征/默认工具时。

K1 = 1

线性特征 X1 里“要估计的系数”的个数。你这里公式是 '0 + prices'，所以只有 1 列（价格）。

MD = 21

工具变量（或矩条件）列数的总计。简单理解就是“参与运算的 IV 列数”，包含 pyblp 自动生成的和你自己加的（比如 demand_instruments20）。

H = 1

内部使用的“人群/节点”数量，这里是 1（无额外分组或随机项时就是 1）。

In [102]:
nl_results = problem.solve(rho = .7)    # rho is the within-nest correlation, we used rammda in the chalss.

Solving the problem ...

Rho Initial Values:
 All Groups  
-------------
+7.000000E-01

Rho Lower Bounds:
 All Groups  
-------------
+0.000000E+00

Rho Upper Bounds:
 All Groups  
-------------
+9.900000E-01

Starting optimization ...



GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                 
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm      Theta    
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1     00:00:00         0             1            0            0          0     +1.331657E+02                 +6.086235E+02  +7.000000E-01
 1     00:00:00         0             2            0            0          0     +4.727024E+01  +8.589549E+01  +1.624078E+01  +9.900000E-01
 1     00:00:00         1             3            0            0          0     +4.720903E+01  +6.120631E-02  +1.143692E-10  +9.824626E-01

Optimization completed after 00:00:01.
Computing the Hessian and updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
G

## 不是一个nest了

In [ ]:
# Nested Logit
product_data1 = product_data.copy()
product_data1['nesting_ids'] = product_data1['mushy'] 
# 新建列 nesting_ids，用现有列 mushy 的取值当分组标签（按 mushy 分“组”）。 mushy 是 Nevo 谷物（cereal）示例数据里的一个产品特征列，表示“容易变糊/在牛奶里容易软”（质地特性），通常是二元变量：0=不易变糊，1=易变糊。
# 含义：这是“多巢”结构的嵌套 Logit，不同 mushy 值的麦片在不同巢里。ρ>0 时，巢内产品更相似、替代更强，放松简单 Logit 的 IIA。

groups = product_data1.groupby(['market_ids','nesting_ids'])
product_data1['demand_instruments20'] = groups['shares'].transform(np.size) # number of products in each nest within each market
nl_formaulation = pyblp.Formulation('0 + prices')
problem = pyblp.Problem(nl_formaulation, product_data1)

nl_results = problem.solve(rho = .7)    # rho is the within-nest correlation, we used rammda in the chalss.

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    K1    MD    H 
---  ----  ---  ----  ----  ---
94   2256   5    1     21    2 

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices
Solving the problem ...

Rho Initial Values:
 All Groups  
-------------
+7.000000E-01

Rho Lower Bounds:
 All Groups  
-------------
+0.000000E+00

Rho Upper Bounds:
 All Groups  
-------------
+9.900000E-01

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                 
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm      Theta    
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1     00:00:00         0             1      